# Re_annotate after subset annotation
### After multiple reannotations on different subsets, transfer all reannotations

In [1]:
import scanpy as sc
import matplotlib.pyplot as plt
import math
import numpy as np
import pandas as pd
import seaborn as sns
import symphonypy as sp
import anndata as ad
from tqdm import tqdm

/home/felix/miniconda3/envs/new_sclitr/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/home/felix/miniconda3/envs/new_sclitr/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/home/felix/miniconda3/envs/new_sclitr/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_hdf instead.
  warnings.warn(msg, FutureWarning)
/home/felix/miniconda3/envs/new_sclitr/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom instead.
  warnings.warn(msg, FutureWarning)
/home/felix/miniconda3/envs/new_sclitr/lib/python3.11/site-packages/anndata/utils.py:429: 

In [ ]:
DATA_PATH = "/placodes/data"

In [3]:
adata_conc = sc.read_h5ad(f"{DATA_PATH}/anndatas/cranial_placodal_integrated_clonal_clusters_annotated.h5ad")

### Transfer the first level annotations

In [4]:
subclustered= sc.read_h5ad(f"{DATA_PATH}/anndatas/reclustered_subset_doublets_removed.h5ad")

In [ ]:
annotations=pd.read_csv(f"{DATA_PATH}/annotations/Annotations_scRNA_clonal - subset.csv")

In [ ]:
annotations = annotations[["re_leiden_2", "annotation"]]

In [8]:
# Give annotation to subclusterd
mapping_dict = annotations.set_index('re_leiden_2')['annotation'].to_dict()
map_dict={}
for x in mapping_dict:
    map_dict[str(x)] = mapping_dict[x]
del mapping_dict
subclustered.obs['reannotation'] = subclustered.obs['re_leiden_2'].map(map_dict)


In [10]:
# Transfer lens
cluster = subclustered[subclustered.obs["re_leiden_2"]=="38"]
tmp_c=cluster[cluster[:,"Cryaa"].X>0].copy()
subclustered.obs.loc[tmp_c.obs_names, "reannotation"] = "Lens"

### Trasnfer to the full data

In [12]:
# Transfer to full data. 
new_name = "new_annotation"
adata_conc.obs[new_name] = ""
for cell in tqdm(adata_conc.obs_names):
    if cell in subclustered.obs_names:
        adata_conc.obs.loc[cell, "new_annotation"] = subclustered.obs.loc[cell, "reannotation"] 
    else:
        adata_conc.obs.loc[cell, "new_annotation"] = adata_conc.obs.loc[cell, "annotation"] 

100%|██████████| 171093/171093 [00:41<00:00, 4168.67it/s]


In [ ]:
# Solve unclear annotaitons
probelmatic_annotations=["right upper part -skin, lower left-otic vesicle"]
take=adata_conc[adata_conc.obs.new_annotation.isin(probelmatic_annotations)].copy()
to_rename = take[take[:,["Pdgfa"]].X > 1].obs_names.tolist()
adata_conc.obs.loc[to_rename, "new_annotation"] = "Epidermis"

In [16]:
# Solve unclear annotaitons
probelmatic_annotations=["Trigeminal+Vestibular"]
take=adata_conc[adata_conc.obs.new_annotation.isin(probelmatic_annotations)].copy()
to_rename = take[take[:,["Pdgfa"]].X > 1].obs_names.tolist()
adata_conc.obs.loc[to_rename, "new_annotation"] = "Trigeminal ganglia neurons"
to_rename = take[take[:,["Pdgfa"]].X < 1].obs_names.tolist()
adata_conc.obs.loc[to_rename, "new_annotation"] = "Vestibular and spiral ganglia neurons"

In [19]:
# Save 
adata_conc.write_h5ad(f"{DATA_PATH}/anndatas/cranial_placodal_reannotated.h5ad")